In [1]:
# bibliotecas básicas
import pandas as pd #biblioteca para manipulação de dados
import numpy as np #biblioteca para utilizacao de vetores e matrizes
import matplotlib.pyplot as plt #bibloteca para plotar graficos

In [2]:
#liberando acesso do colab aos arquivos no drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
#lendo o csv que contem a base de dados e armazanando em um df
df = pd.read_csv('/content/gdrive/My Drive/IGTI/Aulas praticas/phpOJxGL9.csv')

In [3]:
df.head(15)

NameError: ignored

In [ ]:
df.shape

(583, 11)

In [ ]:
df.describe()

,V1,V3,V4,V5,V6,V7,V8,V9,V10,Class
count,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000
mean,44.746141,3.298799,1.486106,290.576329,80.713551,109.910806,6.483190,3.141852,0.947064,1.286449
std,16.189833,6.209522,2.808498,242.937989,182.620356,288.918529,1.085451,0.795519,0.318492,0.452490
min,4.000000,0.400000,0.100000,63.000000,10.000000,10.000000,2.700000,0.900000,0.300000,1.000000
25%,33.000000,0.800000,0.200000,175.500000,23.000000,25.000000,5.800000,2.600000,0.700000,1.000000
50%,45.000000,1.000000,0.300000,208.000000,35.000000,42.000000,6.600000,3.100000,0.947064,1.000000
75%,58.000000,2.600000,1.300000,298.000000,60.500000,87.000000,7.200000,3.800000,1.100000,2.000000
max,90.000000,75.000000,19.700000,2110.000000,2000.000000,4929.000000,9.600000,5.500000,2.800000,2.000000


In [5]:
# precisa fazer tratamento da coluna V2, que é o gênero (M/F)
mapGender = {
    'Female': 0,
    'Male': 1
}

df['V2'] = df['V2'].map(mapGender)

In [6]:
# separar o label do dataset original (coluna de saída, que vai ser o calculo)
labels = np.array(df['Class'])

# armazena a ordem das features
feature_list = list(df.columns)

In [7]:
# remove a coluna de saída do dataset original
df = df.drop('Class', axis=1)

In [ ]:
df.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10'], dtype='object')

In [8]:
# é necessário converter o dataset para array para poder trabalhar com ele daqui pra frente
data = np.array(df)

In [9]:
# faz a divisão entre treinamento e teste
from sklearn.model_selection import train_test_split

train_data, test_data, train_label, test_label = train_test_split(data, labels, test_size = 0.25, random_state = 42)

In [ ]:
df.head(5)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74
2,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89
3,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00
4,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40


Trabalhando com o Random Search

In [12]:
# importa as bibliotecas dos modelos
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

# bibliotecas para tratamento de dados
from scipy.stats import uniform
from scipy.stats import randint

# bibliotecas de medidas
from sklearn.metrics import f1_score, make_scorer

In [ ]:
#definindo o tipo de validacao cruzada e o numero de folds
cv_strat = StratifiedKFold(n_splits = 10)

In [ ]:
#definindo a estrategia de score a partir da metrica f1
f1 = make_scorer(f1_score)

In [ ]:
#definindo os hiperparâmetros que serão calculados
distributions = dict(kernel = ['sigmoid', 'rbf'], 
                     C = uniform(loc=0, scale=10))  # entre 0 e 10

Beleza, agora vamos trabalhar com o SVC, para ele calcular os melhores valores de hiperparâmetros que setamos

In [ ]:
#instânciando 
classifier = SVC()

In [ ]:
# Aqui faz o grid search
random_SVC = RandomizedSearchCV(classifier, distributions, cv=cv_strat, scoring=f1, random_state=42, n_iter=5)
random_SVC.fit(data, labels)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='scale', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=5, n_jobs=None,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2c628bfcf8>,
                                        'kernel': ['sigmoid', 'rbf']},
                   pre_dispatch='2*n_jobs', random_state=42, refit=True,
                   return_train_score=False, scoring=make_scorer(f1_score),
                   verbose=0)

In [ ]:
# resultado completo
random_SVC.cv_results_

{'mean_fit_time': array([0.00872674, 0.01143827, 0.00777454, 0.00797956, 0.01044469]),
 'mean_score_time': array([0.00175998, 0.00175807, 0.00167797, 0.00167763, 0.00196645]),
 'mean_test_score': array([0.73363107, 0.83281808, 0.73623012, 0.73446312, 0.83281808]),
 'param_C': masked_array(data=[3.745401188473625, 1.834347898661638,
                    5.986584841970366, 4.458327528535912,
                    0.5808361216819946],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['sigmoid', 'rbf', 'sigmoid', 'sigmoid', 'rbf'],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 3.745401188473625, 'kernel': 'sigmoid'},
  {'C': 1.834347898661638, 'kernel': 'rbf'},
  {'C': 5.986584841970366, 'kernel': 'sigmoid'},
  {'C': 4.458327528535912, 'kernel': 'sigmoid'},
  {'C': 0.5808361216819946, 'kernel': 'rbf'}],
 'rank_test_score'

In [ ]:
# melhores resultados encontrados pelo Random Search
print('Melhor resultado f1:', random_SVC.best_score_)
print('\n\nMelhor configuração de hiperparâmetros:', random_SVC.best_params_)
print( '\n\nConfigurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: \n', random_SVC.best_estimator_)

Melhor resultado f1: 0.832818081808181


Melhor configuração de hiperparâmetros: {'C': 1.834347898661638, 'kernel': 'rbf'}


Configurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: 
 SVC(C=1.834347898661638, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)


Agora vamos calcular os hiperparâmetros com o Random Forest

In [13]:
#define o numero de folds
cv_strat = StratifiedKFold(n_splits = 10)

In [14]:
#métrica F1
f1 = make_scorer(f1_score)

In [15]:
#configura os hiperparâmetros que serão calculados
distributions_RF = dict(n_estimators = randint(100, 200),
                        bootstrap = [True, False],
                        criterion = ['gini', 'entropy'])

In [16]:
#instancia o modelo
classifier_RF = RandomForestClassifier(random_state=42)

In [17]:
#instancia o random search
random_RF = RandomizedSearchCV(classifier_RF, distributions_RF, cv = cv_strat, n_iter=10, scoring=f1, random_state=42)


In [18]:
random_RF.fit(data, labels)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_s...
                     

In [19]:
# melhores resultados encontrados pelo Random Search
print('Melhor resultado f1:', random_RF.best_score_)
print('\n\nMelhor configuração de hiperparâmetros:', random_RF.best_params_)
print( '\n\nConfigurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: \n', random_RF.best_estimator_)

Melhor resultado f1: 0.8019647527623143


Melhor configuração de hiperparâmetros: {'bootstrap': True, 'criterion': 'gini', 'n_estimators': 171}


Configurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: 
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=171,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
